- Dados: Focos de calor do INPE na frequência anual - https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/EstadosBr_sat_ref/MG/

- Dados: Focos de calor do INPE na frequência mensal - https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/Brasil/

- Código realizado por: Enrique V. Mattos - 11/06/2024 & Modificado por: Natanael Silva Oliveira - 16/12/2024

# **1° Passo:** Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
import os                        
import requests                  
from bs4 import BeautifulSoup   
import re                       
import glob                      
from io import BytesIO
import zipfile

# vamos ignorar vários avisos
import warnings
warnings.filterwarnings('ignore')

# **PARTE 1):** Baixando e organizando os dados de focos de calor

## - Baixando os dados

In [2]:
# URLs base
url_ano_anteriores = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/Brasil_sat_ref/'
url_ano_atual = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/Brasil/'

#================================================================#
#           ANOS ANTERIORES: focos_br_ref_2003.zip    
#================================================================#
# Faz a requisição para pegar os links dos arquivos ZIP
result = requests.get(url_ano_anteriores)
soup = BeautifulSoup(result.content, 'html.parser')
zip_files = soup.find_all('a', href=re.compile("\.zip"))

# Lista para armazenar DataFrames de 2003 a 2023
dataframes_2003_a_2023 = []

for i in zip_files:
    filename = i.get_text().strip()
    url = f'{url_ano_anteriores}{filename}'
    print(f"Baixando e processando: {filename}")

    # Faz o download do arquivo ZIP
    response = requests.get(url)
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        # Extrai e lê o arquivo CSV dentro do ZIP
        for file in z.namelist():
            with z.open(file) as f:
                df = pd.read_csv(f)
                
                # Renomeia colunas para compatibilidade no arquivo de 2023
                if '2023' in filename:
                    df.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)
                
                dataframes_2003_a_2023.append(df)

# Concatena todos os DataFrames em um só
df_2003_a_2023 = pd.concat(dataframes_2003_a_2023, ignore_index=True)

# Remove colunas desnecessárias e ajusta os nomes
df_2003_a_2023.drop(['id_bdq', 'foco_id', 'pais'], axis=1, inplace=True)
df_2003_a_2023.rename(columns={'data_pas': 'data'}, inplace=True)

# Reorganiza as colunas
df_2003_a_2023 = df_2003_a_2023[['data', 'lat', 'lon', 'municipio', 'estado', 'bioma']]

#================================================================#
#            ANO ATUAL: focos_mensal_br_202401.csv	
#================================================================#
result = requests.get(url_ano_atual)
soup = BeautifulSoup(result.content, 'html.parser')
csv_files = soup.find_all('a', href=re.compile("\.csv"))

# Lista para armazenar DataFrames do ano atual
dataframes_2024 = []

for i in csv_files:
    filename = i.get_text().strip()
    url = f'{url_ano_atual}{filename}'
    print(f"Baixando e processando: {filename}")

    # Faz o download e lê o arquivo CSV
    response = requests.get(url)
    df = pd.read_csv(BytesIO(response.content))
    dataframes_2024.append(df)

# Concatena os DataFrames do ano atual
df_2024 = pd.concat(dataframes_2024, ignore_index=True)

# seleciona para o satélite de referência AQUA_M-T
df_2024 = df_2024[df_2024['satelite']=='AQUA_M-T']

# remove colunas
df_2024.drop(['satelite'], axis=1, inplace=True)

# renomeia coluna
df_2024.rename(columns={'data_hora_gmt': 'data'}, inplace=True)

# reposiciona as colunas
df_2024 = df_2024[['data','lat','lon','municipio','estado','bioma']]

#================================================================#
# Mostrando os DataFrames gerados
#================================================================#

df_2003_a_2023

df_2024


Baixando e processando: focos_br_ref_2003.zip
Baixando e processando: focos_br_ref_2004.zip
Baixando e processando: focos_br_ref_2005.zip
Baixando e processando: focos_br_ref_2006.zip
Baixando e processando: focos_br_ref_2007.zip
Baixando e processando: focos_br_ref_2008.zip
Baixando e processando: focos_br_ref_2009.zip
Baixando e processando: focos_br_ref_2010.zip
Baixando e processando: focos_br_ref_2011.zip
Baixando e processando: focos_br_ref_2012.zip
Baixando e processando: focos_br_ref_2013.zip
Baixando e processando: focos_br_ref_2014.zip
Baixando e processando: focos_br_ref_2015.zip
Baixando e processando: focos_br_ref_2016.zip
Baixando e processando: focos_br_ref_2017.zip
Baixando e processando: focos_br_ref_2018.zip
Baixando e processando: focos_br_ref_2019.zip
Baixando e processando: focos_br_ref_2020.zip
Baixando e processando: focos_br_ref_2021.zip
Baixando e processando: focos_br_ref_2022.zip
Baixando e processando: focos_br_ref_2023.zip
Baixando e processando: focos_mens

## - Junta e salva os dados de `2003-2023` com `2024`

In [14]:
%%time
# junta os dataframes dos anos anteriores com o ano atual
df_total = pd.concat([df_2003_a_2023,df_2024], ignore_index=True)

#=================================================#
#                TODAS COLUNAS
#=================================================#
dfx = df_total.copy()

#=================================================#
#                   ESTADOS
#=================================================#
dfx = df_total.copy() ; dfx.drop(['data','lat','lon','municipio','bioma'], axis=1, inplace=True)
dfx.to_csv('estados.csv', compression={'method': 'zip'}, index=False)

#=================================================#
#                   MUNÍCIPIOS
#=================================================#
dfx = df_total.copy() ; dfx.drop(['data','lat','lon','estado','bioma'], axis=1, inplace=True)
dfx.to_csv('municipios.csv', compression={'method': 'zip'}, index=False)

#=================================================#
#                 LATITUDE
#=================================================#
dfx = df_total.copy() 

# multiplica por 10.000 e transforma de real para inteiro para inteiro. Diminuir o tamanho da variável.
dfx['lat'] = dfx['lat']*10000.
dfx['lat'] = dfx['lat'].astype(np.int32)

# salva CSV
dfx.drop(['lon','estado','municipio','bioma'], axis=1, inplace=True)
dfx.to_csv('lat.csv', compression={'method': 'zip'}, index=False)

#=================================================#
#                 LONGITUDE 
#=================================================#
dfx = df_total.copy() 
# multiplica por 10.000 e transforma de real para inteiro. Diminuir o tamanho da variável.
dfx['lon'] = dfx['lon']*10000.
dfx['lon'] = dfx['lon'].astype(np.int32)

# salva CSV
dfx.drop(['data','lat','estado','municipio','bioma'], axis=1, inplace=True)
dfx.to_csv('lon.csv', compression={'method': 'zip'}, index=False)

#=================================================#
#                  BIOMA
#=================================================#
dfx = df_total.copy() 
dfx.drop(['data','lat','lon','estado','municipio'], axis=1, inplace=True)
dfx.to_csv('biomas.csv', compression={'method': 'zip'}, index=False)

CPU times: total: 21.1 s
Wall time: 49.3 s
